In [1]:
print("trying again! was able to download nvidia nemo toolkit and torch libraries until now. facing issues downloading sample audio files for now. ")

trying again! was able to download nvidia nemo toolkit and torch libraries until now. facing issues downloading sample audio files for now. 


In [ ]:
import os, subprocess, sys

def pipi(args: str) -> bool:
    print(">> pip", args)
    r = subprocess.run([sys.executable, "-m", "pip"] + args.split(), check=False)
    if r.returncode != 0:
        print("!! pip failed with code", r.returncode)
    return r.returncode == 0

print("Upgrading pip/setuptools/wheel...")
pipi("install -U pip setuptools wheel")

# Try GPU wheels first; if that fails, fall back to CPU.
print("Installing Torch...")
ok = pipi("install --index-url https://download.pytorch.org/whl/cu124 torch==2.4.0")
if not ok:
    ok = pipi("install torch==2.4.0")

# Remove unneeded packages that often cause conflicts; harmless if not present.
pipi("uninstall -y torchvision jax jaxlib")

print("🔁 Restarting runtime to finalize Torch setup...")
os._exit(0)


Upgrading pip/setuptools/wheel...
>> pip install -U pip setuptools wheel
Installing Torch...
>> pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.4.0
>> pip uninstall -y torchvision jax jaxlib


In [1]:
# NeMo 2.4.1 (no torchaudio path)
!pip install --no-cache-dir --prefer-binary "nemo_toolkit[asr]==2.4.1"

# App deps (no torchaudio): ffmpeg wrapper, soundfile for PCM, gradio for UI, bm25 for search
!pip install --no-cache-dir --prefer-binary ffmpeg-python soundfile gradio rank_bm25 jedi

# System ffmpeg binary
!apt -yqq install ffmpeg


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pyannote-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pyannote-metrics to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pyannote-core to determine which version is compatible with oth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ffmpeg-python]
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [1]:
import torch, numpy
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("numpy:", numpy.__version__)

from nemo.collections.asr.models import ASRModel
print("✅ NeMo import OK")


torch: 2.4.0+cu124 cuda: True
numpy: 1.26.4


[NeMo W 2025-10-26 22:58:43 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-10-26 22:58:43 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-10-26 22:58:43 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-10-26 22:58:43 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


OSError: /usr/local/lib/python3.12/dist-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev

In [2]:
# See if torchaudio is present
!python -c "import importlib.util; print('torchaudio present:', importlib.util.find_spec('torchaudio') is not None)"
!pip show torchaudio || true

# Remove it
!pip uninstall -y torchaudio


torchaudio present: True
Name: torchaudio
Version: 2.8.0+cu126
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough, Moto Hira, Caroline Chen, Jeff Hwang, Zhaoheng Ni, Xiaohui Zhang
Author-email: soumith@pytorch.org
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: torch
Required-by: 
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


In [1]:
import torch, subprocess
print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())

from nemo.collections.asr.models import ASRModel
print("✅ NeMo import OK")

# optional: show ffmpeg version
subprocess.run(["ffmpeg","-hide_banner","-version"], check=False)


Torch: 2.4.0+cu124 CUDA: True
✅ NeMo import OK


CompletedProcess(args=['ffmpeg', '-hide_banner', '-version'], returncode=0)

In [2]:
import torch, numpy
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("numpy:", numpy.__version__)

from nemo.collections.asr.models import ASRModel
print("✅ NeMo import OK")


torch: 2.4.0+cu124 cuda: True
numpy: 1.26.4
✅ NeMo import OK


In [3]:
import sqlite3, datetime as dt
from typing import Optional

DB_PATH = "logs.db"

def get_conn():
    conn = sqlite3.connect(DB_PATH)
    conn.execute(
        "CREATE TABLE IF NOT EXISTS logs (\n"
        "  id INTEGER PRIMARY KEY AUTOINCREMENT,\n"
        "  ts TEXT NOT NULL,\n"
        "  mood TEXT,\n"
        "  tags TEXT,\n"
        "  text TEXT NOT NULL\n"
        ")"
    )
    return conn

def insert_log(text: str, mood: Optional[str] = None, tags: Optional[str] = None, ts: Optional[str] = None):
    conn = get_conn()
    ts = ts or dt.datetime.now().isoformat(timespec="seconds")
    conn.execute("INSERT INTO logs (ts, mood, tags, text) VALUES (?,?,?,?)", (ts, mood, tags, text))
    conn.commit()
    conn.close()
    return ts

def fetch_logs(since: Optional[str] = None, until: Optional[str] = None, mood: Optional[str] = None):
    conn = get_conn()
    q = "SELECT id, ts, mood, tags, text FROM logs WHERE 1=1"
    params = []
    if since: q += " AND ts >= ?"; params.append(since)
    if until: q += " AND ts <= ?"; params.append(until)
    if mood and mood.strip(): q += " AND mood = ?"; params.append(mood.strip())
    q += " ORDER BY ts DESC"
    rows = conn.execute(q, params).fetchall()
    conn.close()
    return rows


In [4]:
import ffmpeg, tempfile, os

def to_16k_mono_wav(in_path: str) -> str:
    if not os.path.exists(in_path):
        raise FileNotFoundError(f"Audio not found: {in_path}")
    fd, out_path = tempfile.mkstemp(suffix=".wav"); os.close(fd)
    try:
        (
            ffmpeg
            .input(in_path)
            .output(out_path, acodec="pcm_s16le", ac=1, ar="16000")
            .overwrite_output()
            .run(quiet=False, capture_stdout=True, capture_stderr=True)
        )
        return out_path
    except ffmpeg.Error as e:
        print("FFmpeg stderr:\n", (e.stderr or b"").decode("utf-8", "ignore"))
        raise


In [5]:
from nemo.collections.asr.models import ASRModel
import torch

MODEL_SMALL = "stt_en_conformer_ctc_small"
MODEL_LARGE = "stt_en_conformer_ctc_large"
use_large = torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory >= 10_000_000_000
model_name = MODEL_LARGE if use_large else MODEL_SMALL

print(f"Loading NeMo ASR model: {model_name}")
asr_model = ASRModel.from_pretrained(model_name=model_name).eval()

def transcribe_with_nemo(path: str) -> str:
    wav16 = to_16k_mono_wav(path)
    texts = asr_model.transcribe(paths2audio_files=[wav16])
    return texts[0] if isinstance(texts, list) and texts else str(texts)


Loading NeMo ASR model: stt_en_conformer_ctc_large
[NeMo I 2025-10-26 23:02:11 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.10.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_2.4.1/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo
[NeMo I 2025-10-26 23:02:13 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-10-26 23:02:19 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2025-10-26 23:02:19 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket1/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket2/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket3/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket4/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket5/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket6/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket7/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    

[NeMo I 2025-10-26 23:02:19 nemo_logging:393] PADDING: 0
[NeMo I 2025-10-26 23:02:21 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.4.1/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.


In [6]:
# If not already installed in this runtime:
!pip install -U google-generativeai --quiet


In [ ]:
import os, textwrap, re, math, json
import google.generativeai as genai

# --- SET YOUR GEMINI KEY HERE ---
# Option A: hardcode (quick tests)
os.environ["GEMINI_API_KEY"] = "PUT_GEMINI_KEY"

# Option B: prompt for it (safer in shared notebooks)
if "GEMINI_API_KEY" not in os.environ or not os.environ["GEMINI_API_KEY"]:
    from getpass import getpass
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY: ")

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Choose a model: "gemini-1.5-flash" (fast, cheap) or "gemini-1.5-pro" (best quality)
GEMINI_MODEL = "gemini-1.5-flash"

generation_config = {
    "temperature": 0.2,
    "top_p": 0.9,
    "top_k": 40,
    "max_output_tokens": 2048,  # bump if needed
}

model = genai.GenerativeModel(GEMINI_MODEL, generation_config=generation_config)

def gcall(system_instructions: str, user_prompt: str):
    # Gemini uses "system_instruction" on the model itself. Easiest: instantiate ad-hoc per call.
    tmp = genai.GenerativeModel(
        GEMINI_MODEL,
        system_instruction=system_instructions,
        generation_config=generation_config,
    )
    resp = tmp.generate_content(user_prompt)
    return resp.text if hasattr(resp, "text") else str(resp)


In [11]:
# Replace this path with your actual file
transcript_text = transcribe_with_nemo("/content/sample_audio.wav")
print("✅ Transcript saved. Length:", len(transcript_text))
print(transcript_text[:1000])  # preview first 1000 chars


FileNotFoundError: Audio not found: /content/sample_audio.wav

In [12]:
transcript_text = transcribe_with_nemo("/content/my_audio.wav")
print(transcript_text[:1000])  # preview first 1000 chars


FileNotFoundError: Audio not found: /content/my_audio.wav

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
!ls -R "/content/drive/MyDrive/Colab Notebooks"


'/content/drive/MyDrive/Colab Notebooks':
00_pytorch_fundamentals.ipynb  NVIDIANEMO.ipynb
content			       Personal_Research_Assistant_Agent.ipynb
Nvidia_nemo_3.ipynb	       PersonalResearchAssistantAgent.ipynb
NVIDIA_NeMo_DailyDiary.ipynb   sample_audio.wav

'/content/drive/MyDrive/Colab Notebooks/content':
sample_audio.wav


In [29]:
import os

path = "/content/drive/MyDrive/Colab Notebooks/content/sample_audio.wav"  # adjust if needed
print("Exists?", os.path.exists(path))


Exists? True


In [31]:
def _hypo_to_text(h):
    # Normalize various return shapes from NeMo
    if isinstance(h, str):
        return h
    if hasattr(h, "text"):              # Hypothesis object
        return h.text
    if isinstance(h, dict):             # sometimes dict-like
        return h.get("text") or h.get("pred_text") or str(h)
    if isinstance(h, (list, tuple)) and h and isinstance(h[0], str):
        return " ".join(h)
    return str(h)

def transcribe_with_nemo(path: str) -> str:
    wav16 = to_16k_mono_wav(path)
    try:
        outs = asr_model.transcribe([wav16], batch_size=1)  # NeMo 2.x signature
    except TypeError:
        outs = asr_model.transcribe([wav16])
    # outs is usually a list with one item
    first = outs[0] if isinstance(outs, (list, tuple)) and outs else outs
    return _hypo_to_text(first)


In [32]:
import os

path = "/content/drive/MyDrive/Colab Notebooks/content/sample_audio.wav"  # adjust if needed
print("Exists?", os.path.exists(path))


Exists? True


In [33]:
transcript_text = transcribe_with_nemo(path)  # keep your 'path' variable as before
print("✅ Transcript saved. Length:", len(transcript_text))
print(transcript_text[:1000])


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  8.56it/s]

✅ Transcript saved. Length: 210
the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle tastes fine with ham tacos al pastor are my favorite a zestful food is the hot cross bun


In [ ]:
# --- 7. Summarize transcript with Google Gemini (robust + non-exp models, graceful 429 handling) ---
!pip -q install -U google-generativeai

import os, re, json, time, textwrap
import google.generativeai as genai
from google.api_core import exceptions as gcore

# ========== 7A) Configure your API key ==========
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") or "PUT_GEMINI_KEY"
assert GEMINI_API_KEY and GEMINI_API_KEY != "YOUR_GEMINI_KEY_HERE", "Add your Gemini API key (or set env var GEMINI_API_KEY)."
genai.configure(api_key=GEMINI_API_KEY)

# ========== 7B) Ensure transcript exists ==========
assert 'transcript_text' in globals() and isinstance(transcript_text, str) and transcript_text.strip(), \
    "transcript_text not found. Run Step 6 first to create it."

# ========== 7C) Pick a supported (non-experimental) model that can generateContent ==========
PREFERRED = [
    "gemini-1.5-flash-002",
    "gemini-1.5-flash-8b",
    "gemini-1.5-flash",
    "gemini-1.5-pro-002",
    "gemini-1.5-pro",
]

def pick_model():
    models = list(genai.list_models())
    def supports_generate(m):
        methods = getattr(m, "supported_generation_methods", None) or []
        return "generateContent" in methods
    def is_non_exp(name: str) -> bool:
        # Keep to GA (non "exp", "research") names for better availability
        lowered = name.lower()
        return ("exp" not in lowered) and ("research" not in lowered)

    candidates = {m.name for m in models if supports_generate(m) and is_non_exp(m.name)}
    for short in PREFERRED:
        full = f"models/{short}"
        if full in candidates:
            return short

    # Last resort: any non-exp generate-capable model
    for full in candidates:
        return full.split("/", 1)[-1]

    # If we got here, your key likely has the API disabled or no models accessible
    raise RuntimeError(
        "No accessible Gemini model with generateContent for this API key. "
        "Enable 'Generative Language API' in your Google Cloud project and verify model access."
    )

MODEL_NAME = pick_model()
print("Using Gemini model:", MODEL_NAME)
model = genai.GenerativeModel(MODEL_NAME)

# ========== 7D) Helpers (chunking + call with backoff) ==========
def chunk_text(s: str, max_chars: int = 12000):
    s = s.strip()
    if len(s) <= max_chars:
        return [s]
    out, i = [], 0
    while i < len(s):
        j = min(len(s), i + max_chars)
        k = s.rfind(".", i, j)
        if k == -1 or k <= i + int(0.6 * max_chars):
            k = j
        out.append(s[i:k].strip())
        i = k
    return out

def call_gemini(prompt: str, max_retries: int = 2):
    """
    Call generate_content with simple exponential backoff on 429.
    Surface clear guidance if quota is zero.
    """
    wait = 2.0
    last = None
    for attempt in range(max_retries + 1):
        try:
            resp = model.generate_content(prompt)
            return getattr(resp, "text", "") or ""
        except gcore.ResourceExhausted as e:
            # 429 / quota exceeded
            msg = str(e)
            if "limit: 0" in msg or "free_tier" in msg:
                raise RuntimeError(
                    "Gemini quota appears to be zero for this key/project. "
                    "Enable billing or use a key with quota. "
                    "Alternatively set USE_LOCAL_FALLBACK=True below to run a local summarizer."
                )
            last = e
            time.sleep(wait)
            wait *= 2
        except gcore.NotFound as e:
            # Model not visible on this endpoint -> repick once
            raise RuntimeError(
                f"Model '{MODEL_NAME}' not available to this key. "
                "Try a different model or project. Underlying error: " + str(e)
            )
        except Exception as e:
            last = e
            time.sleep(wait)
            wait *= 2
    raise last

system_style = (
    "You are a concise notes assistant. Be faithful to the transcript. "
    "Prefer bullet points. Keep names/times if present. Avoid hallucinations."
)

def per_chunk_prompt(chunk: str) -> str:
    return f"""{system_style}

Summarize this transcript chunk with ≤8 bullet points and a 1-sentence gist.
Return Markdown.

<TRANSCRIPT_CHUNK>
{chunk}
</TRANSCRIPT_CHUNK>
"""

def final_prompt(joined_bullets: str) -> str:
    return f"""{system_style}

You are given bullet summaries of transcript chunks.
1) Merge into one clean summary (bullets, grouped by theme).
2) List action items with owners and due dates if stated.
3) Extract 5–8 keywords (comma-separated).
4) Propose a short title (≤60 chars).
5) ALSO return a compact JSON object with:
   {{
     "title": "...",
     "summary": ["...","..."],
     "action_items": [{{"owner":"", "task":"", "due":""}}],
     "keywords": ["...", "..."]
   }}

Chunks to merge:
{joined_bullets}
"""

# Replace your previous extract_json_block with this one:

def extract_json_block(s: str):
    # 1) Prefer a fenced ```json ... ``` block
    m = re.search(r"```json\s*(\{.*?\})\s*```", s, flags=re.DOTALL | re.IGNORECASE)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass

    # 2) Fallback: scan for the last balanced {...} and try to parse it
    last_obj = None
    stack = 0
    start = None
    for i, ch in enumerate(s):
        if ch == "{":
            if stack == 0:
                start = i
            stack += 1
        elif ch == "}":
            if stack > 0:
                stack -= 1
                if stack == 0 and start is not None:
                    candidate = s[start:i+1]
                    try:
                        last_obj = json.loads(candidate)
                    except Exception:
                        pass
    return last_obj



# ========== 7E) Optional local fallback (set True to use if quota==0) ==========
USE_LOCAL_FALLBACK = False

def summarize_locally(text: str) -> str:
    """
    Lightweight heuristic fallback: make bullets from sentences.
    (No heavy model downloads; meant to work if you have no Gemini quota.)
    """
    import re
    sents = [s.strip() for s in re.split(r'[.!?]\s+', text) if s.strip()]
    bullets = "\n".join(f"- {s}" for s in sents[:12])
    title = (sents[0][:60] + "…") if sents else "Transcript Summary"
    json_block = {
        "title": title,
        "summary": sents[:8],
        "action_items": [],
        "keywords": []
    }
    return f"# {title}\n\n{bullets}\n\n```json\n{json.dumps(json_block, ensure_ascii=False, indent=2)}\n```"

# ========== 7F) Run summarization ==========
chunks = chunk_text(transcript_text, max_chars=12000)
print(f"Gemini: processing {len(chunks)} chunk(s) with {MODEL_NAME}")

try:
    chunk_bullets = []
    for i, ch in enumerate(chunks, 1):
        print(f"  • Summarizing chunk {i}/{len(chunks)}...")
        out = call_gemini(per_chunk_prompt(ch), max_retries=2)
        chunk_bullets.append(f"### Chunk {i}\n{out}")

    joined = "\n\n".join(chunk_bullets)
    final_md = call_gemini(final_prompt(joined), max_retries=2)

    print("\n===== Final Summary (Markdown + JSON at bottom) =====\n")
    print(final_md)

    summary_json = extract_json_block(final_md)
    if summary_json:
        print("\n\n===== Parsed JSON =====")
        print(json.dumps(summary_json, indent=2, ensure_ascii=False))
    else:
        print("\n(Note) Could not parse JSON from the model output. You can still copy the Markdown above.)")

except RuntimeError as e:
    # Clear guidance for zero-quota scenarios, plus optional fallback
    print("Gemini call failed:", e)
    if USE_LOCAL_FALLBACK:
        print("\nRunning local fallback (heuristic bullets, no LLM)…\n")
        print(summarize_locally(transcript_text))
    else:
        print("\nSet USE_LOCAL_FALLBACK=True in this cell to get a quick local summary without using Gemini.")


Using Gemini model: gemini-2.5-flash-preview-09-2025
Gemini: processing 1 chunk(s) with gemini-2.5-flash-preview-09-2025
  • Summarizing chunk 1/1...

===== Final Summary (Markdown + JSON at bottom) =====

**Sensory Observations and Food Preferences**

### Summary
*   **Sensory/Temperature Effects:**
    *   The stale smell of old beer lingers.
    *   Heat is required to bring out the beer odor.
    *   A cold dip restores health and zest.
*   **Food Preferences:**
    *   Tacos al pastor are the speaker's favorite.
    *   A salt pickle tastes fine with ham.
    *   The hot cross bun is noted as a zestful food.

### Action Items
(None stated)

### Keywords
Beer smell, heat, cold dip, zest, tacos al pastor, salt pickle, hot cross bun

### Proposed Title
Sensory Observations and Food Preferences

---

```json
{
  "title": "Sensory Observations and Food Preferences",
  "summary": [
    "The stale smell of old beer lingers; heat is required to bring out the odor.",
    "A cold dip restor

In [40]:
summary_json = extract_json_block(final_md)
if summary_json:
    print("\n\n===== Parsed JSON =====")
    import json
    print(json.dumps(summary_json, indent=2, ensure_ascii=False))
else:
    print("\n(Note) Could not parse JSON from the model output. You can still copy the Markdown above.)")




===== Parsed JSON =====
{
  "title": "Sensory Observations and Food Preferences",
  "summary": [
    "The stale smell of old beer lingers; heat is required to bring out the odor.",
    "A cold dip restores health and zest.",
    "Tacos al pastor are the speaker's favorite.",
    "Preferred foods include a salt pickle with ham and the hot cross bun (noted as zestful)."
  ],
  "action_items": [],
  "keywords": [
    "Beer smell",
    "heat",
    "cold dip",
    "zest",
    "tacos al pastor",
    "salt pickle",
    "hot cross bun"
  ]
}


In [ ]:
# ===== Step 8: Gradio App (Upload → Transcribe (NeMo) → Summarize (Gemini)) =====
import os, tempfile, subprocess, json, re
import gradio as gr

# ---- 0) Config: your Gemini key (set it once per runtime) ----
# If you've already set this earlier, you can skip this line.
os.environ.setdefault("GEMINI_API_KEY", "PUT_GEMINI_KEY")  # <-- replace

# ---- 1) Safe JSON extractor (from our last fix) ----
def extract_json_block(s: str):
    m = re.search(r"```json\s*(\{.*?\})\s*```", s, flags=re.DOTALL | re.IGNORECASE)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass
    last_obj = None
    stack = 0
    start = None
    for i, ch in enumerate(s):
        if ch == "{":
            if stack == 0: start = i
            stack += 1
        elif ch == "}":
            if stack > 0:
                stack -= 1
                if stack == 0 and start is not None:
                    candidate = s[start:i+1]
                    try:
                        last_obj = json.loads(candidate)
                    except Exception:
                        pass
    return last_obj

# ---- 2) ffmpeg helper: to 16k mono WAV (no torchaudio needed) ----
def to_16k_mono_wav(in_path: str) -> str:
    if not os.path.exists(in_path):
        raise FileNotFoundError(f"Audio not found: {in_path}")
    fd, out_path = tempfile.mkstemp(suffix=".wav"); os.close(fd)
    cmd = [
        "ffmpeg", "-y", "-i", in_path,
        "-ac", "1", "-ar", "16000",
        "-f", "wav", out_path
    ]
    run = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if run.returncode != 0:
        raise RuntimeError(f"ffmpeg failed:\n{run.stderr.decode('utf-8', errors='ignore')}")
    return out_path

# ---- 3) Load NeMo once (reuse if already loaded) ----
try:
    asr_model
except NameError:
    from nemo.collections.asr.models import ASRModel
    import torch
    MODEL_SMALL = "stt_en_conformer_ctc_small"
    MODEL_LARGE = "stt_en_conformer_ctc_large"
    use_large = torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory >= 10_000_000_000
    model_name = MODEL_LARGE if use_large else MODEL_SMALL
    print(f"[Gradio] Loading NeMo ASR model: {model_name}")
    asr_model = ASRModel.from_pretrained(model_name=model_name).eval()

def nemo_transcribe(path: str) -> str:
    wav16 = to_16k_mono_wav(path)
    # NeMo >=2.x: EncDecCTCModel.transcribe() returns list[Hypothesis] or similar
    hyps = asr_model.transcribe([wav16], batch_size=1, return_hypotheses=True)
    hyp = hyps[0]
    # Prefer text if present
    text = getattr(hyp, "text", None) or str(hyp)
    return text

# ---- 4) Gemini caller (v1beta REST via google-ai-generativelanguage) ----
# If you already have call_gemini() in your notebook, we use it; else define a compact one.
try:
    call_gemini
except NameError:
    from google.ai.generativelanguage_v1beta import GenerativeServiceClient
    from google.ai.generativelanguage_v1beta.types import content as glm

    def call_gemini(prompt: str, model: str = "gemini-2.5-flash-preview-09-2025", retries: int = 1):
        api_key = os.environ.get("GEMINI_API_KEY", "")
        if not api_key:
            raise RuntimeError("GEMINI_API_KEY is not set. Set it in the cell above.")
        client = GenerativeServiceClient(api_key=api_key)
        req = glm.GenerateContentRequest(
            model=f"models/{model}",
            contents=[glm.Content(parts=[glm.Part(text=prompt)])],
            generation_config=glm.GenerationConfig(temperature=0.2),
        )
        last_err = None
        for _ in range(max(1, retries+1)):
            try:
                resp = client.generate_content(req)
                txt = ""
                if resp.candidates and resp.candidates[0].content and resp.candidates[0].content.parts:
                    txt = "".join(p.text or "" for p in resp.candidates[0].content.parts)
                return txt.strip()
            except Exception as e:
                last_err = e
        raise last_err

# ---- 5) Prompt template for summarization ----
def build_summary_prompt(transcript: str, user_goal: str = "") -> str:
    goal = user_goal.strip() or "Provide a concise bullet summary, action items (if any), 6–10 keywords, and a short title."
    return f"""
You are given a transcript. {goal}

Return two sections:
1) A human-friendly Markdown summary (bullets OK).
2) A fenced ```json block with keys: title, summary (array of strings), action_items (array), keywords (array).

Transcript:
\"\"\"{transcript.strip()}\"\"\"
"""

# ---- 6) Gradio pipeline ----
GEMINI_MODEL_CHOICES = [
    "gemini-2.5-flash-preview-09-2025",
    "gemini-2.0-flash",
    "gemini-1.5-flash",
    # add more if you have access
]

def ui_pipeline(audio_file, do_summary, gemini_model, custom_goal):
    if audio_file is None:
        return "", "", ""

    # 1) Transcribe
    transcript = nemo_transcribe(audio_file)

    md_summary = ""
    parsed_json = ""
    # 2) Optional: summarize with Gemini
    if do_summary:
        prompt = build_summary_prompt(transcript, custom_goal or "")
        try:
            md_summary = call_gemini(prompt, model=gemini_model, retries=2)
            js = extract_json_block(md_summary)
            parsed_json = json.dumps(js, indent=2, ensure_ascii=False) if js else "(No JSON parsed)"
        except Exception as e:
            md_summary = f"**Gemini error:** `{type(e).__name__}` – {e}"
            parsed_json = ""

    return transcript, md_summary, parsed_json

with gr.Blocks(title="NeMo + Gemini (no torchaudio)") as demo:
    gr.Markdown("## Speech → Text (NeMo) ➜ (Optional) Summary (Gemini)\nUpload audio, transcribe with NeMo, and summarize with Gemini.\n")

    with gr.Row():
        audio_in = gr.File(label="Upload audio (wav/mp3/m4a/ogg/etc.)")

    with gr.Accordion("Gemini (optional)", open=False):
        do_sum = gr.Checkbox(True, label="Summarize with Gemini")
        model_dd = gr.Dropdown(GEMINI_MODEL_CHOICES, value="gemini-2.5-flash-preview-09-2025", label="Gemini model")
        custom_goal = gr.Textbox(label="(Optional) Custom summary goal / instructions", placeholder="e.g., extract action items and a 1-line executive summary")

    with gr.Row():
        transcript_out = gr.Textbox(lines=10, label="Transcript (NeMo)", show_copy_button=True)
    with gr.Row():
        md_out = gr.Markdown(label="Gemini Summary (Markdown)")
    with gr.Row():
        json_out = gr.Code(language="json", label="Parsed JSON from summary")

    run_btn = gr.Button("Transcribe (and Summarize)")
    run_btn.click(
        fn=ui_pipeline,
        inputs=[audio_in, do_sum, model_dd, custom_goal],
        outputs=[transcript_out, md_out, json_out],
    )

# In Colab: share=True gives you a public link.
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56f583d0a9ce2e87f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
